In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import DBLPScholarStructuredBenchmark

benchmark = DBLPScholarStructuredBenchmark(data_dir_path="../data/")
benchmark

17:14:44 INFO:Extracting DBLP-Scholar-Structured...
17:14:44 INFO:Reading DBLP-Scholar-Structured row_dict...
17:14:45 INFO:Reading DBLP-Scholar-Structured train.csv...
17:14:45 INFO:Reading DBLP-Scholar-Structured valid.csv...
17:14:45 INFO:Reading DBLP-Scholar-Structured test.csv...


<DBLPScholarStructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/DBLP-GoogleScholar/dblp_scholar_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'authors', 'venue', 'year']

In [6]:
from tqdm.auto import tqdm
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'semantic_title': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'authors': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'venue': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'semantic_venue': {
        'source_attr': 'venue',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'year': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=benchmark.row_dict)
row_numericalizer.attr_info_dict

17:14:46 INFO:For attr=title, computing actual max_str_len
17:14:47 INFO:For attr=title, using actual_max_str_len=30
17:14:47 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:14:51 INFO:For attr=authors, computing actual max_str_len
17:14:51 INFO:For attr=authors, using actual_max_str_len=30
17:14:51 INFO:For attr=venue, computing actual max_str_len
17:14:51 INFO:For attr=venue, using actual_max_str_len=30
17:14:52 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:14:55 INFO:For attr=year, computing actual max_str_len
17:14:55 INFO:For attr=year, using actual_max_str_len=8


{'title': NumericalizeInfo(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=30, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_title': NumericalizeInfo(source_attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 

In [11]:
datamodule = benchmark.build_pairwise_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=20,
    row_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

17:14:55 INFO:GPU available: True, used: True
17:14:55 INFO:TPU available: None, using: 0 TPU cores
17:14:55 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(model, datamodule)

17:14:57 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 26.3 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
10.2 M    Trainable params
16.1 M    Non-trainable params
26.3 M    Total params


17:14:58 WARNING:true_pair_set has 3066 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [15]:
model.blocker_net.get_signature_weights()

{'title': 0.3843172788619995,
 'semantic_title': 0.3145996034145355,
 'authors': 0.16635939478874207,
 'venue': 0.06752988696098328,
 'semantic_venue': 0.03342347964644432,
 'year': 0.033770330250263214}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.16656279214708633,
 'valid_f1_at_0.5': 0.6955665024630542,
 'valid_f1_at_0.7': 0.9347728295096717,
 'valid_f1_at_0.9': 0.919607843137255,
 'valid_pair_entity_ratio_at_0.3': 6.176377952755906,
 'valid_pair_entity_ratio_at_0.5': 1.036745406824147,
 'valid_pair_entity_ratio_at_0.7': 0.605249343832021,
 'valid_pair_entity_ratio_at_0.9': 0.5091863517060368,
 'valid_precision_at_0.3': 0.09085500594934558,
 'valid_precision_at_0.5': 0.5362025316455696,
 'valid_precision_at_0.7': 0.9011274934952298,
 'valid_precision_at_0.9': 0.9670103092783505,
 'valid_recall_at_0.3': 0.9990654205607477,
 'valid_recall_at_0.5': 0.9897196261682243,
 'valid_recall_at_0.7': 0.9710280373831776,
 'valid_recall_at_0.9': 0.8766355140186916}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.16640698036771578,
  'test_f1_at_0.5': 0.7164582629266645,
  'test_f1_at_0.7': 0.9551282051282052,
  'test_f1_at_0.9': 0.9203980099502487,
  'test_pair_entity_ratio_at_0.3': 6.1634363541121004,
  'test_pair_entity_ratio_at_0.5': 0.9895233106338397,
  'test_pair_entity_ratio_at_0.7': 0.5835515976951283,
  'test_pair_entity_ratio_at_0.9': 0.4924044002095338,
  'test_precision_at_0.3': 0.09077001529831719,
  'test_precision_at_0.5': 0.5611434621492853,
  'test_precision_at_0.7': 0.9362657091561939,
  'test_precision_at_0.9': 0.9840425531914894,
  'test_recall_at_0.3': 0.9981308411214953,
  'test_recall_at_0.5': 0.9906542056074766,
  'test_recall_at_0.7': 0.974766355140187,
  'test_recall_at_0.9': 0.8644859813084113}]